In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [8]:
df_1 = df.filter(df.total_votes>=20)
df_1.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   13446431|R1QXNQBTH7LIUB|B00V03PK74|     159528888|Kalamera freestan...|Major Appliances|          3|            9|         22|   N|                Y|They claim that i...|They claim that i...| 2015-08-31|
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|    

In [9]:
df_2 = df_1.filter((df_1.helpful_votes/df_1.total_votes)>=0.5)
df_2.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|           83|         89|   N|                Y|Warning! Long Rev...|Where do I begin?...| 2015-08-31|
|         US|   13254956| R5XKK92G5N9FU|B00EM2JPLW|     824392206|Samsung NE58F9500...|Major Appliances|          1|    

In [10]:
paid_df= df_2.filter(df_2.vine == "Y")
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   31302915|R20O97P5V9CZ21|B00P7QI4IM|     901651668|Whynter BR-130SB ...|Major Appliances|          3|           21|         27|   Y|                N|Nice but had some...|I was very excite...| 2015-06-07|
|         US|   38320678| RCNB7YGLWKCV5|B00NQFSSWS|     692404913|Samsung Counter T...|Major Appliances|          5|    

In [11]:
unpaid_df= df_2.filter(df_2.vine == "N")
unpaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|           83|         89|   N|                Y|Warning! Long Rev...|Where do I begin?...| 2015-08-31|
|         US|   13254956| R5XKK92G5N9FU|B00EM2JPLW|     824392206|Samsung NE58F9500...|Major Appliances|          1|    

In [12]:
paid_reviews = paid_df.count()
paid_reviews

35

In [13]:
unpaid_review = unpaid_df.count()
unpaid_review

4957

In [14]:
paid_5star = paid_df.filter(paid_df.star_rating == 5).count()
paid_5star

18

In [15]:
paid_5star_pct = (paid_5star/paid_reviews) * 100
paid_5star_pct

51.42857142857142

In [17]:
unpaid_5star = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_5star

1963

In [18]:
unpaid_5star_pct = (unpaid_5star/unpaid_review) * 100
unpaid_5star_pct

39.60056485777688